## Ortho Prediction Software Developing by krish:
#### Check all Function Working better or Not

#### main / Global Variable use in this program

In [ ]:
import time , os, sys,subprocess , numpy as np , pandas as pd 

In [ ]:
#Blastp = /usr/bin/blastp
Blastp = "blastp"
Blastp_data =  './blastp_data/'
cpu_count = 1 # Default value
#genome = None
genome = ""
infinite_loop = 70
inflation_factor = 1.4
#mode = None
mode = None
Cluster_out = "./cluster_out/"
#threshold_score : 0
threshold_score =0
Save_raw_blastp_score = False
Score_file = "./score_file"
Species = "./species/"
verbose = False

In [ ]:
import glob
a = glob.glob("species/""*")
a[1].split("\\")[1]

## 1. GetMatrixNumber()

In [ ]:
#The Following matrix files must be in Folder
import os
def GetMatrixNumber():
    "This Function will Return the Matrix name choosed by User"
    import sys
    print ("\n1. BLOSUM45\n2. BLOSUM62\n3. BLOSUM82\n4. Quit")

    metrix_num = input("\nEnter a matrix number: ")
    if metrix_num not in ("1","2",'3'):
        print("Wrong input *%s*Sorry not in list\n"%metrix_num,"*"*20,"Good Bye","*"*20,"\n")
        sys.exit(1)
    if metrix_num =="1":
        return "BLOSUM45"
    if metrix_num == "2":
        return "BLOSUM62"
    if metrix_num == "3":
        return "BLOSUM82"
#GetMatrixNumber()

### 2. GetQuerySequence(genome)

In [ ]:
species = "./species/"
def GetQuerySequence(genome):
    """This Function Read Fastaq Files and return as a list Format with gene Position as a index
    Later I will reduce code Lengths and Speed"""
    gene_sequence=""
    gene_sequence_list= []
    try:
        with open(species+genome) as gene:
            for each_line in gene:
                if ">" in each_line:
                    if gene_sequence != "":                        
                        gene_sequence_list.append(gene_sequence)
                        gene_sequence = ""
                gene_sequence = gene_sequence+each_line
            gene_sequence_list.append(gene_sequence)                   
        return gene_sequence_list                        
                                            
    except IOError as err:
        print ("IOError occurred in GetQuerySequence function : " + str(err)    )
b = GetQuerySequence("AAE")

In [ ]:
b[0]

## 3.Write Query(query, parallel_num)

In [ ]:
def WriteQuery(query, parallel_num):
    
    try:
        with open("./query"+"_"+str(parallel_num)+".csv", "w") as write_query:
            write_query.write(query)
    except IOError as err:
        print ("IOError occurred in WriteQuery function : " + str(err))

In [ ]:
WriteQuery("krish is from ",10)

## 4. RunBlast( Subject , parallel_num)

In [ ]:
import subprocess
blastp_matrix = "BLOSUM62"
def RunBlast(subject, parallel_num):
    "By this Function it will create a Pipe line to run Blastp in Computer by input Parameter"
    subject = Species+subject       
    run_blastp =subprocess.Popen([Blastp, "-query", str(parallel_num), "-subject", subject,
                                "-matrix", blastp_matrix, "-outfmt", "10 qseqid sseqid score length"],
                                 stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    run_blastp_stream = run_blastp.communicate()
    run_blastp_output_stream = run_blastp_stream[0]
    run_blastp_error_stream = run_blastp_stream[1]  
    
    display(run_blastp)
    display(run_blastp_error_stream)
    return run_blastp_output_stream
#RunBlast("AAE","AAE")

## Get_Same_Species_Forward_Best_Hit(blastp_score):

In [ ]:
def Get_Same_Species_Forward_Best_Hit(blastp_score):
    blastp_score_split_list  = []
    temp_best_score = ["-1",'-1','-1']
    second_temp_best_score = []
    best_score = []
    second_best_score = []
    blastp_score_split = blastp_score.split('\n')
    del blastp_score_split[-1] #delete of [''] in last index 

## 21. Read_Species_List()

In [ ]:
species

In [ ]:
import glob
"if "
species = "./species/"
def Read_Species_List(pr=0):
    global species
    species = species
    """ If pr is 1, it will print "Species_List". """ 
    read_species =glob.glob(species+"*")   
    selected_species_dic = {}
    backward_selected_species_dic = {}
    #number = 0
    for i, species in enumerate(sorted(read_species), start=1):
        selected_species_dic[i] = species.split('\\')[-1]
        backward_selected_species_dic[species.split('\\')[-1]] = i 
        if pr == 1 :
            print (str(i)+".", species.split('\\')[-1])
        number = i
    return selected_species_dic, backward_selected_species_dic, number
#Read_Species_List(1)

In [ ]:
def read_species(pr =1):
    import os
    read_species = os.listdir("species/")
    for i,species in enumerate(read_species,start =1):
        print(i,species)
#read_species()

## 22. Del_File(path,file)

In [ ]:
def Del_File(path,file):
    import subprocess
    
    del_file = subprocess.Popen(["del "+path+file],stdout=subprocess.PIPE, stderr = subprocess.PIPE,shell= True)
    del_file_stream = del_file.communicate()
    if not del_file_stream[1]:
        print("Done to del"+path+file)
    elif del_file_stream[1]:
        print(del_file_stream[1])

#### i developed the below Function to remove the file may be different but will check later 

In [ ]:
def del_(file):
    import os
    try:
        os.remove(file)
        print("Successfully_remove",file)
    except:
        print("Check the File And Path")

In [ ]:
del_("krish")

In [ ]:
#!rm krish
#import os
#print(os.listdir())
#! touch krish


### 23.Check_File(file)

In [ ]:
def Check_File(file):
    "Check the file weather exist or not"
    file_list = glob.glob(file+'*')    
    if (Cluster_out+"_geneID_S"+str(threshold_score)+"_"+str(inflation_factor) or Cluster_out+"_KO_ID_S"+str(threshold_score)+"_"+str(inflation_factor)) in file_list:
        print ("Please, set other name of output.")
        sys.exit(2)

In [ ]:
#Check_File()

## Main Program Testing

In [ ]:
# arg parse is not done here
# If parameter not Provided the below code will run

In [ ]:
#if not sys.argv[1:]

import sys
import multiprocessing

# If Program not Passed with Parameter
species = "./species/"  # if Not verified Conflict arise for local and global Variable
print ("1. BLASTP. \n2. BLASTP using precalculated data. \n3. Clustering.\n")
mode = input(">> Select a mode or modes (1 or 2 or 1 3 or 2 3): ") 
selected_species_dic, backward_selected_species_dic, number_i = Read_Species_List(pr =2)
selected_number = input(">> Select Genomes to detect Orthologs(e.g. 1 2 3 4 5 or 1-5) : ")


if selected_number.find("-")>0:
    SN = selected_number.split("-")
    if int(SN[-1])>number_i:
        print("Your input must be less than" , number_i)
        sys.exit(2)
    else:
        user_selected_number = range(int(SN[0]),int(SN[-1])+1)
        print(user_selected_number)
        for j in user_selected_number:
            print(selected_species_dic[j],end=" ")
        print("Are Selected")
else:
    user_selected_number = sorted(set([int(read_species) for read_species in selected_number.split()]))
    if int(user_selected_number[-1])>number_i:
        print ("\nWrongInput\nInput must be less than",number_i)
        sys.exit(2)
    else:
        for j in user_selected_number:
            print(selected_species_dic[j], end =" ")

blastp_matrix = GetMatrixNumber()
cpu_count = int(input("You can use %s processors. \nIf you input >2, The Program will run a parallel--"\
                      %multiprocessing.cpu_count()+"Enter the number of CPU you want to use :"))
if "3" in mode:
    inflation_factor = input("Enter the inflation factor to cluster")
    Cluster_out = input("Set the name of Clustering output :")
    

##### if Paramater Passed with Python Code

In [ ]:
##elif sys.argv[1:]
##command_options = argparse_value
#genomes = command_options.genomes
#mode = command_options.mode
#cpu_count= command_options.cpu_count
#blastp_matrix = command_options.blastp_matrix
#inflation_factor = command_options.inflation_factor
#selected_species_dic,backward_selected_species_dic, number_i = Read_Species_List()
#user_selected_number = [backward_selected_species_dic[ele] for ele in genomes]
#Cluster_out = command_options.Cluster_out

In [ ]:
#Species = command_options.Species
#Blastp = COmmand_options.Blastp
#Score_file = command_options.Score_file
#Blastp_data = command_options.Blastp_data
#save_raw_blastp_score = command_options.threshold_score
#threshold_score = command_options.threshold_score
#verbose = command_options.verbose
#infinite_loop = command_options.infinite_loop
#
#if "3" in mode:
#    Check_File(Cluster_out)
#Del_File(score_file, "*")
#if "3" in mode:
#    Log_file_name = Cluster_out+"_S"+str(threshold_score)+"_"+str(inflation_factor)+".log"
#elif not "3" in mode:
#    Log_file_name = "Log"
#
#with open(Log_file_name, "w") as log:
#    log.write(str(datetime.date_time.now()))
#    log.write("\nmode")
#    for i in mode:
#        log.write(" "+i)
#    log.write("\ngenomes:")
#    for i in user_selected_number:
#        log.write(selected_species_dic[i]+" ")
#    log.write("\nCPU_Count:"+str(cpu_count))
#    log.write("\nblastp matrix :"+blastp_matrix)
#    if "3" in mode:
#        log.write("\ninflation_factor: "+str(inflation_factor))
#        log.write("\nCluster out:"+Cluster_out)
#    log.write("\nSpecies :"+Species)
#    log.write("\nBlastp: "+Blastp)
#    log.write("\nScore file : "+ Score_file)
#    log.write("\nBlastp_data : "+ Blastp_data)
#    log.write("\nsave rawblastp score : "+ str(save_raw_blastp_score))
#    log.write("\n")
#

In [ ]:
start_time_OBH = time.time()

In [ ]:
if "1" in  mode:
    backward_best_hit_work_list = Oneway_Threshold_Best_Hit(mode)
    pool = multiprocessing.Pool(cpu_count)
    results = pool.map(Backward_Best_Hit, backward_best_hit_work_list)
    pool.close()
    pool.join()
elif "2" in mode:
    used_precalculated_data_list = []
    new_calculated_data_list = []
    precalculated_data_list = glob.glob(Blastp_data+"*oneway_threshold_best_hit_S"+str(threshold_score))
    print(precalculated_data_list)
    backward_best_hit_work_list = Oneway_Threshold_Best_Hit(mode)
    
    if not backward_best_hit_work_list ==[]:
        #If backward_best_hit_work_list is an empty list, pool instance can't finish the work.
        pool = multiprocessing.Pool(cpu_count)
        results = pool.map(Backward_Best_Hit, backward_best_hit_work_list)
        pool.close()
        pool.join()
    else:
        results = [0,0]
Del_File("./","query*")
finish_time_OBH = time.time()
blastp_time_log = float(((finish_time_OBH - start_time_OBH)/60))
print("BLASTP searches + forward best Hit + backwardbest hit took %f minutes"%blastp_time_log)

with open(Log_file_name , "a") as log:
    log.write("Backward_Best_Hit took "+str(max(results))+" minutes\n")
    log.write("BLASTP + Best_Hit + backward_best_hit searches took"+ str(blastp_time_log)+" minutes\n")
if "3" in mode:
    start_time_clustering = time.time()
    #generate matrix calculate the matrix using MCL algorithm and cluster the Ortholog
    
    print("\n >>> Start MCL algorithm and Clustering ortholog <<<")
    equal_BBH_data = []
    unequal_BBH_data = []
    equal_BBH_data_dic = {}
    second_equal_BBH_data = []
    results = que.Queue()
    tasks = queue.Queue()
    cluster_count = 1
    ortholog_count = 0
    gene_id_dic = {}
    
    with open("myva=gb","r") as id_read:
        #myvba=gb is a database
        for i in id_read:
            gene_name,gene_id = i.split()
            gene_id_dic[gene_id.replace("\n","")] = gene_name # remove '\n'
            
        if "1" in mode:
            for i in user_selected_number:
                for k in user_selected_number:
                    if k<i:
                        pass
                    elif i ==k:
                        Read_Equal_BBH(Score_file+selected_species_dic[i]+"_"+selected_species_dic[k])
                    elif i !=k:
                        Read_Unequal_BBH(Score_file+selected_species_dic[i]+"_"+selected_species_dic[k])
        elif "2" in mode:
            for used_data in used_precalculated_data_list:
                first,second = used_data.split("_")
                if first == second:
                    Read_Equal_BBH(Blastp_data+used-data)
                elif first != second:
                    Read_Unequal_BBH(Blastp_data+used_data)
            for new_data in new_calculated_data_list:
                first,second = new_data.split("_")
                if first== second:
                    Read_Equal_BBH(Score_file+new_data)
                elif first != second:
                    Read_Unequal_BBH(Score_file+new_data)
        matched_BBH_data = []
        matched_BBH_element_data_set = []
        
        for unequal_RBH_element in unequal_BBH_data:
            Matching_BBH(unequal_RBH_element)
            temp_results_list = []
            if results._qsize() !=0: #return the number of results as Queue.
                while not results.empty():
                    get_results = results.get()
                    temp_results_list.append(get_results)
                matched_BBH_data.append(temp_results_list)
        bar = Bar("Processing", max= len(matched_BBH_data))
        for data in matched_BBH_data:
            Generating_Matrix_Clustering_Ortholog(data, bar)
        bar.finish()
        finish_time_clustering = time.time()
        mcl_time_log = float((finish_time_clustering - start_time_clustering)/60)
        remark_time_log = float((finish_time_clustering- start_time_OBH)/60)
        print("MCL algorithm and Ortholog Clustering took %.2f minutes"%remark_time_log)
        
        if "3" in mode:
            with open(Log_file_name,"a") as log:
                log.write("Ortholog Count: "+str(ortholog_count)+","+"Cluster count: "+ str(cluster_count-1)+"\n")
                log.write("MCL algorithm and Ortholog Clustering took "+str(mcl_time_log)+"minutes\n")
                log.write("xxx Program took "+str(remark_time_log)+"minutes\n")

In [ ]:
print(multiprocessing.Pool(2))